In [1]:
%load_ext sql
%sql sqlite:///PS2.db

Problem Set #2
=======

### Deliverables:

Submit your answers using the `submission_template.txt` file that is posted on the class website. Follow the instructions on the file! Upload the file at Canvas (under PS2).


### Instructions / Notes:

**_Read these carefully_**

* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* _Have fun!_

Problem 1: Verifying Functional Dependencies [12 points]
---------

For this part, you will need to provide a _single_ SQL query which will check whether a certain condition holds on the **hospital** table in the provided database:

In [2]:
%sql select * from hospital limit 2

 * sqlite:///PS2.db
Done.


provider,hospital,address,city,state,zip,county,phone_number,hospital_type,hospital_owner,emergency_service,condition,measure_code
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-CARD-2
10018,CALLAHAN EYE FOUNDATION HOSPITAL,1720 UNIVERSITY BLVD,BIRMINGHAM,AL,35233,JEFFERSON,2053258100,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Surgical Infection Prevention,SCIP-INF-1


You need to evaluate any requested conditions in the following way: **your query should return an empty result if and only if the condition holds on the instance.**  If the condition doesn't hold, your query should return something non-empty, but it doesn't matter what this is.

Note our language here: the conditions that we specify cannot be proved to hold **in general** without knowing the externally-defined functional dependencies; so what we mean is, _check whether they **are not violated** for the provided instance_.

You may assume that there are no `NULL` values in the tables.

### Part (a)  [7 points]

Is $\{provider\}$ a **superkey** for relation $Hospital$?

In [ ]:
%%sql
SELECT count(*) as res
FROM hospital
GROUP BY provider
HAVING res > 1;

### Part (b) [5 points]

Does $\{Zip\} \rightarrow \{City, State\}$ hold for relation $Hospital$?

In [4]:
%%sql 
WITH single AS (
	SELECT distinct zip, city, state
	FROM hospital
	GROUP BY zip)
SELECT *
FROM single
GROUP BY zip
HAVING count(*) > 1;

 * sqlite:///PS2.db
Done.


zip,city,state


Problem 2: Superkeys & Decompositions [20 points]
---------

Consider a relation $S(A,B,C,D,E,F)$ with the following functional dependencies:

* $\{A\} \rightarrow \{D\}$
* $\{A\} \rightarrow \{E\}$
* $\{D\} \rightarrow \{C\}$
* $\{D\} \rightarrow \{F\}$

In each part of this problem, we will examine different properties the provided schema.

To answer **yes**, provide python code that assigns the variable ```answer``` to ```True``` and assigns ```explanation``` to be a python string which contains a (short!) explanation of why.  For example:

```python
answer = True
explanation = "All keys are superkeys."
```

To answer **no**, provide python code that assigns the variable ```answer``` to ```False``` and assigns ```explanation``` to be a python string which contains a (short!) explanation of why.  For example:

```python
answer = False
explanation = "D is not a superkey because its closure is {D,C,F}."
```

### Part (a) [4 points]

Is it correct that ${A,B}$ is a superkey?

In [ ]:
answer = True
explanation = """Closure of A = {A, D, E, C, F}, Closure of B = {B}, together, with armstrongs axiom 2, the closure is {A,B,C,D,E,F}."""

### Part (b) [4 points]

Is it correct that the decomposition $ABC$, $CDE$, $EFA$ is lossless-join?

In [ ]:
answer = False
explanation = """Using Chase Algo: R1 = a, b, c, d1, e1, f1, R2 = a2, b, c, d, e, f2, R3 = a, b3, c3, d3, e, f
it ends in R1 = a, b, c, d1, e, f, R2 = a2, b2, c, d, e, f2, R3 = a, b3, c, d1, e, f.
None of the resulting rows are without subscripts, therefore it's not lossless.
"""

### Part (c) [4 points]

Is it correct that the decomposition $ABC$, $CDE$, $EFA$ is dependency preserving?

In [ ]:
answer = False
explanation = """Because functional dependency A -> D is not perserved in any of the decomposition."""

### Part (d) [4 points]

Is the functional dependency $\{A\} \rightarrow \{E,F\}$ logically implied by FDs present in the relation?

In [ ]:
answer = True
explanation = """Because A implies D and D implies F. A also implies E directly."""

### Part (e) [4 points]

Is it correct that relation $S$ is in BCNF? 

In [23]:
answer = False
explanation = """A -> D is a nontrivial FD, yet A is not a superkey. That is because the closure of A does not include B."""

Problem 3: Relational Algebra [18 points]
---------

Consider the following relational schema for conference publications:
*  `Article(artid, title, confid, numpages)`
*  `Conference(confid, name, year, location)`
*  `Author(artid, pid)`
*  `Person(pid, name, affiliation)`

Express the following queries in the extended Relational Algebra (you can also use the aggregation operator if necessary). To write the RA expression, use the LaTex mode that ipython notebook provides. For example:

$$\pi_{name}(\sigma_{affiliation="UW-Madison"}(Person))$$ 

### Part (a) [4 points]

Output the name of every person affiliated with `UW-Madison` who has submitted an article in 2019.

$$\pi_{name}((\sigma_{affiliation="UW-Madison"}(Person)\bowtie Author) \bowtie (\sigma_{year=2019}(Conference) \bowtie Author))$$

### Part (b) [5 points]

Output the names of the people who coauthored an article with `John Doe`. Be careful: a person cannot be coauthor with herself!

$$(\pi_{pid}((Author / \sigma_{name="John Doe"}(Person)) \bowtie Author) \bowtie Person) - (\sigma_{name="John Doe"}(Person))$$

### Part (c) [4 points]

Count how many people published an article in the conference `SIGMOD` in 2018, but not in 2019. 

$$\pi_{name}(((\sigma_{name="sigma", year = "2018"}(Conference)\bowtie Article)\bowtie Author) \bowtie Person) - \pi_{name}(((\sigma_{name="sigma", year = "2019"}(Conference)\bowtie Article)\bowtie Author) \bowtie Person)$$

### Part (d) [5 points]

Translate the following SQL query to Relational Algebra.

In [ ]:
%%sql
SELECT pid, COUNT(A.artid)
FROM Article A, Conference C, Author U
WHERE A.confid = C.confid AND C.name = "SIGMOD" AND U.artid = A.artid
GROUP BY pid ;

$$\gamma_{pid,COUNT(artid)}((\sigma_{name="SIGMOD"}(Conference))\bowtie Article)\bowtie Author $$